## Finding Postcodes of Australia

If you ever struggle to find postcodes within striking distance of where you are, here is a simple tool everyone can use in order to locate geographically where any postcode is. Note that some of thes reflect Australia Post Boxes (what they call as 'PO Box'), so even they have been captured here as well.

### **1. Import Packages**

The main packages you need for this exercise are *pandas* and *geopy*. If you do find other useful packages to uplift this, please advise.

In [86]:
import pandas as pd
from geopy.distance import geodesic

### **2. Load Postcode dataset**

Included is a dataset with all of the Australian Postcodes with the Locality Name, State, Longitude and Latitude (the latters have been abbreviated as 'long' and 'lat' respectively). Please note that this is an exhaustive list of postcodes in Australia that have existed since 1967, which was also when Postcodes were introduced in Australia. 

In [88]:
df = pd.read_csv('australian_postcodes_with_latlon.csv')
print(df)

       postcode                        locality state        long        lat
0           200                             ANU   ACT  149.119000 -35.277700
1           200  Australian National University   ACT  149.118900 -35.277700
2           800                          DARWIN    NT  130.836680 -12.458684
3           800                     DARWIN CITY    NT  130.836680 -12.458684
4           801                          DARWIN    NT  130.836680 -12.458684
...         ...                             ...   ...         ...        ...
18521      9013                        BRISBANE   QLD  152.823141 -27.603479
18522      9015                        BRISBANE   QLD  152.823141 -27.603479
18523      9464                    NORTHGATE MC   QLD  153.074982 -27.397055
18524      9726                   GOLD COAST MC   QLD  153.412197 -28.008783
18525      9999                      NORTH POLE   VIC  144.956776 -37.817403

[18526 rows x 5 columns]


### **3. Identify the Latitude and Longitude of the Australian City Centres**

The dataset and the citation of these Latitude and Longitudes is courtesy https://www.matthewproctor.com/australian_postcodes. Please feel free to visit in case you wanted more exposure to the way in which the data is captured.

In [90]:
city_centres = {
    'Sydney': (-33.8688, 151.2093),
    'Melbourne': (-37.8136, 144.9631),
    'Brisbane': (-27.4698, 153.0251),
    'Perth': (-31.9505, 115.8605),
    'Adelaide': (-34.9285, 138.6007),
    'Canberra': (-35.2809, 149.1300),
    'Hobart': (-42.8821, 147.3272),
    'Darwin': (-12.4634, 130.8456)
}

### **4. Establish function to find Postcodes within 40 km of a given city**

Note that the specific business case being tackled involved finding the postcodes within 40 km of Sydney, but you can tweak this to any city, and if required, you can tweak this to any geographic location, but we are keeping this within Australia.

In [92]:
def postcodes_within_distance(city):
    if city not in city_centres:
        raise ValueError("Invalid city.")
    centre = city_centres[city]
    df['distance_km'] = df.apply(lambda row: geodesic(centre, (row['lat'], row['long'])).km, axis=1)
    result = df[df['distance_km'] <= 40].sort_values('distance_km') # Change the 'distance_km' as required in your business case.

    # Save to CSV
    result.to_csv(f"postcodes_within_40km_{city}.csv", index=False)
    print(f"Exported to postcodes_within_40km_{city}.csv")

    return result

### **5. Pick the Australian City of Interest**

Upon printing below, the output is also exported into a CSV format, which you can then use for your analyses.

In [94]:
postcodes_within_distance("Sydney")

Exported to postcodes_within_40km_Sydney.csv


,postcode,locality,state,long,lat,distance_km
6151,2891,SYDNEY GATEWAY FACILITY,NSW,151.209000,-33.868800,0.027758
6150,2891,INTERNATIONAL MC,NSW,151.209000,-33.868800,0.027758
605,1230,QUEEN VICTORIA BUILDING,NSW,151.206708,-33.871749,0.405601
604,1229,QUEEN VICTORIA BUILDING,NSW,151.206708,-33.871749,0.405601
603,1228,QUEEN VICTORIA BUILDING,NSW,151.206708,-33.871749,0.405601
...,...,...,...,...,...,...
5310,2765,BOX HILL,NSW,150.860258,-33.663002,39.579881
5309,2765,BERKSHIRE PARK,NSW,150.860258,-33.663002,39.579881
920,2083,CHEERO POINT,NSW,151.192044,-33.510467,39.777306
5287,2759,ERSKINE PARK,NSW,150.781767,-33.822520,39.900397


An additional perk is that you can see the exact distance as well, as you try and validate the distance. Moreover, it does not factor in the actual travel time. For example Cheero Point in NSW is actually 53 km from Sydney CBD when we take this by road. However this tool only factors the straight-line path (i.e. **as the crow flies**).